# Point Cloud Segmentation

In [ ]:
This notebook aims at coding a single step to process point cloud, i.e. regroup all from SuperpointInference.ipynb in a single class. The method will take the point cloud in input and will o